In [1]:
### setup code, don't change! ###
import mani_skill.envs
import gymnasium as gym
import numpy as np
from mani_skill.utils.visualization import images_to_video
from mani_skill.utils.wrappers.record import RecordEpisode
import mplib
from tqdm.notebook import tqdm
from transforms3d.euler import euler2quat, quat2euler
from test_env import TestTaskEnv

env = gym.make("TestTask-v1",
                num_envs=1,
                control_mode="pd_joint_pos",
                render_mode="rgb_array",
                reward_mode="none",
                human_render_camera_configs=dict(shader_pack="default"),
                obs_mode="state"
    )
env = RecordEpisode(env, output_dir="drop_cube", video_fps=20, info_on_video=True, save_trajectory=False)
env.reset(seed=42)
robot = env.unwrapped.agent.robot
link_names = [link.get_name() for link in robot.get_links()]
joint_names = [joint.get_name() for joint in robot.get_active_joints()]
planner = mplib.Planner(
    urdf=env.unwrapped.agent.urdf_path,
    srdf=env.unwrapped.agent.urdf_path.replace(".urdf", ".srdf"),
    user_link_names=link_names,
    user_joint_names=joint_names,
    move_group="panda_hand_tcp",
    # ensures planned motions do not exceed these limits
    joint_vel_limits=np.ones(7) * 0.8,
    joint_acc_limits=np.ones(7) * 0.8,
)
# this sets the planner object up such that you can plan with poses in the world frame, which is the default frame of all pose data
# in our simulator
planner.set_base_pose(np.concatenate([robot.pose.sp.p, robot.pose.sp.q]))

/home/zhw/miniconda3/envs/robolearning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


def pick_cube_mp_solution(env):
    robot = env.unwrapped.agent.robot
    panda_hand_tcp = env.unwrapped.agent.robot.links_map["panda_hand_tcp"]
    cube = env.unwrapped.cube
    bin = env.unwrapped.bin
    goal = env.unwrapped.goal_region
   
    ### Your code goes here ###
    grasp_ori = panda_hand_tcp.pose.q[0].numpy().copy()
    z_offset = 0.15
    grasp = 1

    # cube pose in world frame
    cube_p = cube.pose.p[0].numpy().copy()
    bin_p = bin.pose.p[0].numpy().copy()
    goal_p = goal.pose.p[0].numpy().copy()
    grasp_point = bin_p + np.array([0.08, 0.00, 0.0])
   

    planner.update_attached_sphere(
        radius=1.0,
        pose=np.concatenate([panda_hand_tcp.pose.p[0].numpy(), panda_hand_tcp.pose.q[0].numpy()]),
        link_id=panda_hand_tcp.get_index()
    )
    
    # grasp the cube
    grasp_result = planner.plan_screw(
        np.concatenate([grasp_point + np.array([0,0,z_offset]), grasp_ori]),
        robot.qpos.cpu().numpy()[0],
        time_step=env.unwrapped.control_timestep,
        use_attach=True,
        wrt_world=True
    )
    for pos in grasp_result["position"]:
        env.step(np.concatenate([pos, [grasp]]))

    grasp_result = planner.plan_screw(
        np.concatenate([grasp_point, grasp_ori]),
        robot.qpos.cpu().numpy()[0],
        time_step=env.unwrapped.control_timestep,
        use_attach=True,
        wrt_world=True
    )
    for i, pos in enumerate(grasp_result["position"]):
        if i == len(grasp_result["position"]) - 1:
            grasp = -1
        env.step(np.concatenate([pos, [grasp]]))

    # move to bin
    grasp_result = planner.plan_screw(
        np.concatenate([goal_p + np.array([0,0,z_offset]), grasp_ori]),
        robot.qpos.cpu().numpy()[0],
        time_step=env.unwrapped.control_timestep,
        use_attach=True,
        wrt_world=True
    )
    for pos in grasp_result["position"]:
        env.step(np.concatenate([pos, [grasp]]))

    grasp_result = planner.plan_screw(
        np.concatenate([goal_p + np.array([0,0,z_offset]), grasp_ori]),
        robot.qpos.cpu().numpy()[0],
        time_step=env.unwrapped.control_timestep,
        use_attach=True,
        wrt_world=True
    )
    for i, pos in enumerate(grasp_result["position"]):
        if i == len(grasp_result["position"]) - 1:
            grasp = 1
        env.step(np.concatenate([pos, [grasp]]))
    
    # extra steps to see outcome
    for _ in range(5):
        env.step(np.concatenate([robot.qpos.cpu().numpy()[0][:7], [grasp]]))


    ### your code goes above ###

### evaluation code below ###
EPISODES = 10
successes = 0
for i in range(10):
    env.reset(seed=i)
    pick_cube_mp_solution(env)
    success = env.get_info()["success"].item()
    successes += success
env.reset()
print(f"Success rate: {successes/EPISODES}")

/home/zhw/miniconda3/envs/robolearning/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_info to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_info` for environment variables or `env.get_wrapper_attr('get_info')` that will search the reminding wrappers.
  logger.warn(


KeyError: 'position'